In [2]:
%load_ext autoreload
%autoreload 2

# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = 'all'

In [3]:
import numpy as np
import json
from random import sample

import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import torch
import torch.nn.functional as F

from transformers import *

In [5]:
models = {}

In [6]:
model_class, tokenizer_class, shortcut, mask_token = RobertaForMaskedLM, RobertaTokenizer, 'roberta-base', '<mask>'
model, tokenizer = model_class.from_pretrained(shortcut), tokenizer_class.from_pretrained(shortcut)
models[shortcut] = (model, tokenizer, mask_token)

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
model_class, tokenizer_class, shortcut, mask_token = RobertaForMaskedLM, RobertaTokenizer, 'roberta-large', '<mask>'
model, tokenizer = model_class.from_pretrained(shortcut), tokenizer_class.from_pretrained(shortcut)
models[shortcut] = (model, tokenizer, mask_token)

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at roberta-large and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
model, tokenizer, mask_token = models['roberta-large']

In [9]:
text = 'Paris is the _ of France'
text = 'Paris is the captital of'
text = 'Q: 4 and 9, which is larger? A:'
text = '''Q: 3 and 5, which is larger? A: 5.
Q: 4 and 9, which is larger? A:'''
text = '''Q: 5, 3, 3, which is different from the others? A: 5.
Q: 2, 2, 7, which is different from the others? A: 7.
Q: 9, 4, 9, which is different from the others? A:'''
text = '''Q: 5, 3, 2. Which is in the middle? A: 3.
Q: 5, 4, 2. Which is in the middle? A:'''
texts = [
'''\
Q: double c. A: c c.
Q: double b. A:''',
'''\
Q: b b b, c c. Which is more, b or c? A: b.
Q: e e , f f f. Which is more, e or f? A: f.
Q: g g g, d d. Which is more, g or d? A:''',
'''\
a b c changes to a b d.
p q r changes to p q s.
i j k changes to''',
'''\
a b c changes to a b d.
p q r s changes to p q r t.
i j k l m changes to''',
    
    
    
# '''\
# i i j j k k changes to i i j j l l.
# a a b b c c changes to a a b b d d.
# p p q q r r changes to''',
# '''\
# a b c changes to a b d.
# e f g h changes to e f g i.
# u v w x y changes to''',
# '''\
# Q: b b b, c c. Which is more, former or latter? A: former.
# Q: e e , f f f. Which is more, former or latter? A: latter.
# Q: g g g, d d. Which is more, former or latter? A:''',
# '''\
# Q: Surround f with b. A: b f b.
# Q: Surround a with d. A: d a d.
# Q: Surround c with e. A:''',
# '''\
# Q: c, b, f. Which letter follows c? A: b.
# Q: b, a, g. Which letter follows a? A: g
# Q: f, g, d. Which letter follows f? A:''',
]

# text = '''Q: 5, 3, 3, which is special? A: 5.
# Q: 2, 2, 7, which is special? A: 7.
# Q: 9, 4, 9, which is special? A:'''

# text = '''Q: 1, 0, 3, 0, 0, 5. Remove 0s. A: 1, 3, 5.
# Q: 0, 1, 3, 0, 5, 5. Remove 0s. A: 1, 3, 5.
# Q: 2, 0, 0, 4, 1, 0. Remove 0s. A:'''

# text = '''Q: 4, 9, 5, which is the largest? A: 9.
# Q: 8, 2, 6, which is the largest? A: 8.
# Q: 3, 1, 7, which is the largest? A: 7.
# Q: 1, 7, 3, which is the largest? A:'''

# text = '''Q: Swap 4 and 7. A: 7 and 4.
# Q: Swap 9 and 3. A: 3 and 9.
# Q: Swap 5 and 2. A:'''

# text = '''Q: 4 = 4? A: Yes.
# Q: 6 = 2? A: No.
# Q: 2 = 2? A: Yes.
# Q: 5 = 5? A:'''

# text = '''Q: 4 and 4. A: Same.
# Q: 6 and 2. A: Different.
# Q: 2 and 2. A: Same.
# Q: 2 and 2. A:'''

# text = '''Q: Which is the former of 5 and 2? A: 5.
# Q: Which is the latter of 7 and 9? A: 9.
# Q: Which is the latter of 1 and 7? A: 7.
# Q: Which is the latter of 6 and 3? A:'''  # former is wrong

# text = '''Q: 5 and 2, which is the former? A: 5.
# Q: 7 and 9, which is the latter? A: 9.
# Q: 1 and 7, Which is the latter? A: 7.
# Q: 6 and 3. Which is the latter? A:'''

# text = '''Q: 5, 3, 2. Which is the first? A: 5.
# Q: 7, 9, 2. Which is the second? A: 9.
# Q: 1, 4, 7. Which is the third? A: 7.
# Q: 5, 4, 2. Which is the second? A:'''

# text = '''Q: 5, 3, 3. How many 3s? A: 2.
# Q: 4, 2, 7. How many 4s? A: 1.
# Q: 1, 1, 1. How many 1s? A: 3.
# Q: 2, 4, 2. How many 5s? A: 0.
# Q: 6, 1, 6. How many 6s? A:'''

text = texts[-1]

In [12]:
texts = [
    'The ball does not fit into the box because the _ is larger than the _ .',
    'The box can not hold the ball because the _ is small .',
    'John borrowed money from Mary because she was rich .',
    'John borrowed money from Mary because he was poor .',
    'John beat Mary because he was strong .',
    'John beat Mary because she was weak .',
#     'John borrowed from Mary because she was rich .',
    'John surpassed Mary because he was fast .',
#     'John surpassed Mary because she was slow .',
    'Thing X _broke thing Y because thing X was _brittle .',
    'John is _taller than Mary means that Mary is _shorter than John .',
    'John is _older than Mary means that Mary is _younger than John .',
    'John is _stronger than Mary means that Mary is _weaker than John .',
    'John is _old but Mary is _young .',
    'John is old but Mary is _ .',
    'John is rich but Mary is _ .',
    'John is tall but Mary is _ .',
#     'Husky is a kind of _ .',
#     'Lark is a kind of _ .',
#     'Ball A is * bigger than ball B and ball A is * bigger .',
#     'Ball A is * bigger than ball B so ball B is * smaller .',
#     'Ball A is * bigger than ball B and ball A is * heavier .',
#     'Ball A is * bigger than ball B so ball B is * lighter .',
#     'A * dog is a kind of * animal .',
#     'An * apple is a kind of * fruit .',
#     'What is a * dog ? It is a kind of * animal .',
#     'What is an * apple ? It is a kind of * fruit .',
]
templates = [
    'A * %s is a kind of * %s .',
    'A %s is a kind of %s .',
#     'What is an * %s ? It is a kind of * %s .',
]
pairs = [
    ('dog', 'animal'),
    ('banana', 'fruit'),
    ('hammer', 'tool'),
    ('gun', 'weapon'),
#     ('dog', '_'),
#     ('banana', '_'),
#     ('hammer', '_'),
#     ('gun', '_'),
]
# text = texts[-4: -2]
all_attn_probs, idx_pairs, tokenses = [], [], []


# for text in texts[-3:]:
for text in texts[-1:]:
# for pair in pairs[-4:]:
#     text = templates[1] % pair

    print('text1:')
    print(text)
    
    
    if mask_token is not None:
        text = text.replace(' _ ', ' %s ' % mask_token)
    print('text2:')
    print(text)
    
    
    tokens = tokenizer.convert_ids_to_tokens(tokenizer.encode(text, add_special_tokens=True))
    print('token1:')
    print(tokens)
    
    
    tokens = ['*' if token in ['*', 'Ġ*'] else token for token in tokens]
    print('token2:')
    print(tokens)
    
    print('--------------------------------------------------------')
    
    marker = '*'
    
    if marker in tokens:
        print('marker in tokens...')
        # 如果 * 在 tokens 中，
        assert tokens.count(marker) == 2, str(tokens)
        p, h = [i for i, token in enumerate(tokens) if token == marker]
        print(marker)
        print(p,h)
        tokens = [token for token in tokens if token != marker]
        h -= 1
        print(tokens[p], tokens[h])
        
        
        
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    print('token_ids')
    print(token_ids)
    
    # pred_idx 是 <mask> 在句子中的位置
    pred_idx = [i for i, token in enumerate(tokens) if token == mask_token] if mask_token is not None else [-1]
    print(pred_idx)
    
    # 如果 token 不以 G 开头，并且不是 <s> </s> <mask> 则在前面加上 @ ，否则将 G 替换为 空
    tokens = ['@' + token if not token.startswith('Ġ') and token not in ['<s>', '</s>', '<mask>'] else token.replace('Ġ', '') 
              for token in tokens] 
    print(tokens)


    print('#################################################')
    
    input_ids = torch.tensor([token_ids])
    with torch.no_grad():
        # 不追踪网络参数中的导数的目的，总之是为了减少可能存在的计算和内存消耗
        
        
        logits, attns = model(input_ids, output_attentions=True)
        
        print('logits:')
        print(logits)
        
        #print('attentions:')
        #print(attns)
        
        # 对n维输入张量运用Softmax函数，将张量的每个元素缩放到（0,1）区间且和为1。
        # dim:指明维度，dim=0表示按列计算；dim=-1是对某一维度的行进行softmax运算
        probs = F.softmax(logits, dim=-1)
        
        print('probs')
        print(probs)

    for i in pred_idx:
        print(i)
        top_probs, top_indexes = probs[0][i].topk(5)
        
        print('probs[0][i]')
        print(probs[0][i])
        
        print('probs[0][i].topk(5)')
        print(probs[0][i].topk(5))
        
        
        top_tokens = tokenizer.convert_ids_to_tokens(top_indexes)
        print(top_tokens)
        print(top_probs)

    attn_scores, attn_probs = zip(*attns)
    attn_scores, attn_probs = torch.cat(attn_scores, dim=0), torch.cat(attn_probs, dim=0)
    all_attn_probs.append(attn_probs)
    idx_pairs.append((p, h))
    tokenses.append(tokens)
    token_pairs.append((tokens[p].replace('Ġ', ''), tokens[h].replace('Ġ', '')))



text1:
John is tall but Mary is _ .
text2:
John is tall but Mary is <mask> .
token1:
['<s>', 'John', 'Ġis', 'Ġtall', 'Ġbut', 'ĠMary', 'Ġis', '<mask>', 'Ġ.', '</s>']
token2:
['<s>', 'John', 'Ġis', 'Ġtall', 'Ġbut', 'ĠMary', 'Ġis', '<mask>', 'Ġ.', '</s>']
--------------------------------------------------------
token_ids
[0, 10567, 16, 6764, 53, 2708, 16, 50264, 479, 2]
[7]
['<s>', '@John', 'is', 'tall', 'but', 'Mary', 'is', '<mask>', '.', '</s>']
#################################################
logits:
tensor([[[64.1458, -5.7664, 59.5445,  ...,  1.5092,  1.4614, 33.3519],
         [36.0186, -3.9361, 50.4558,  ...,  1.0343,  0.7829, 24.4841],
         [36.0185, -4.1961, 49.6261,  ...,  2.0924,  2.1780, 25.6546],
         ...,
         [43.8663, -4.6278, 52.8192,  ...,  1.2955,  2.3893, 28.9979],
         [46.1004, -4.0114, 57.8426,  ...,  2.0919,  2.3034, 29.4621],
         [59.1389, -4.9138, 70.4290,  ...,  2.8684,  2.0703, 32.7099]]])
probs
tensor([[[9.2314e-01, 4.0065e-31, 9.2668e-03,

NameError: name 'p' is not defined